**Description:**

    基于 2023 年 1 月各站点后台 Payment 数据，统一规范不同站点各字段，多表匹配输出利润模型。
    
<a name=toc></a> **TOC:**
- [一、准备工作](#1)
    - [第三方库准备](#1.1)
    - [原始文件准备](#1.2)
- [二、北美站整合 PST (太平洋标准时间 UTC-8)](#2)
    - [美国站](#2.1)
    - [加拿大](#2.2)
- [三、日本站整合 JST (日本标准时间 UTC+9)](#3)
- [四、欧洲站整合 UTC (世界协调时间)](#4)
    - [德国站](#4.1)
    - [西班牙](#4.2)
    - [法国站](#4.3)
    - [意大利](#4.4)
- [x] [五、全站点整合](#5)
- [ ] [六、利润模型构建](#6)

### <a name="1"></a> 一、准备工作

[回到目录](#toc)

#### <a name="1.1"></a> 第三方库准备

In [1]:
import glob
import io
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

import dateparser
import chardet
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.sans-serif']=['Microsoft YaHei']

#### <a name="1.2"></a> 原始文件准备

In [2]:
list_csv = glob.glob("C:/Users/alldo/Desktop/tmp/Jan-23-利润核算" + '/*.csv')
list_csv

['C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\BCRH-CA_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\BCRH-JP_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\BCRH-US_20231月1-20231月31CustomUnifiedTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-CA_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-DE_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-ES_20231月MonthlyTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-FR_20231月MonthlyTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-IT_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\LX-US_20231月1-20231月31CustomUnifiedTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\ODLF-DE_20231月MonthlyTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23-利润核算\\ODLF-ES_20231月1-20231月31CustomTransaction.csv',
 'C:/Users/alldo/Desktop/tmp/Jan-23

In [3]:
print("文件数量：", len(list_csv))

文件数量： 18


### <a name="2"></a> 二、北美站整合

[回到目录](#toc)

#### <a name="2.1"></a> 美国站

In [4]:
list_cols_us = ["date_time", "settlement_id", "type", "order_id", "sku", "description", "quantity", "marketplace", 
                "account_type", "fulfillment", "order_city", "order_state", "order_postal", "tax_collection_model", 
                "product_sales", "product_sales_tax", "shipping_credits", "shipping_credits_tax", "gift_wrap_credits", 
                "giftwrap_credits_tax", "Regulatory_Fee", "Tax_On_Regulatory_Fee", "promotional_rebates", 
                "promotional_rebates_tax", "marketplace_withheld_tax", "selling_fees", "fba_fees", 
                "other_transaction_fees", "other", "total"]

date_parser_us = lambda x: dateparser.parse(x, settings={'DATE_ORDER': 'MDY'})

df_us_bcrh = pd.read_csv(list_csv[2], names=list_cols_us, skiprows=8, thousands=',', decimal='.', 
                         parse_dates=['date_time'], date_parser=date_parser_us)
df_us_lx = pd.read_csv(list_csv[8], names=list_cols_us, skiprows=8, thousands=',', decimal='.',
                       parse_dates=['date_time'], date_parser=date_parser_us)
df_us_odlf = pd.read_csv(list_csv[14], names=list_cols_us, skiprows=8, thousands=',', decimal='.',
                         parse_dates=['date_time'], date_parser=date_parser_us)
df_us_pj = pd.read_csv(list_csv[16], names=list_cols_us, skiprows=8, thousands=',', decimal='.', 
                       parse_dates=['date_time'], date_parser=date_parser_us)

df_us = pd.concat([df_us_bcrh, df_us_lx, df_us_odlf, df_us_pj])
df_us

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,account_type,fulfillment,...,Regulatory_Fee,Tax_On_Regulatory_Fee,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2023-01-01 17:48:41-08:00,17196053011,Order,111-2105463-8945057,T1028-US2,"ALLDOCUBE Android 11 Tablet 10.1 inch, Smile X...",1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,-10.00,0,-8.40,-9.60,0.00,0.0,0.0,110.39
1,2023-01-01 21:41:05-08:00,17196053011,NaN,18179849-31f1-4131-a212-fff1511724ce,NaN,Save $20 on iPlay 50 tablet,8.0,Amazon.com,Standard Orders,NaN,...,0.0,0,0.00,0,0.00,0.00,0.00,-4.8,0.0,-4.80
2,2023-01-01 22:39:46-08:00,17196053011,NaN,5da09c12-9b6a-42ed-8485-eb19be65fe3f,NaN,Save $20 on Smile X Tablet,1.0,Amazon.com,Standard Orders,NaN,...,0.0,0,0.00,0,0.00,0.00,0.00,-0.6,0.0,-0.60
3,2023-01-01 22:40:41-08:00,17196053011,Order,111-6830145-0975449,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,-7.64,0,-12.35,-10.40,0.00,0.0,0.0,119.59
4,2023-01-01 23:15:44-08:00,17196053011,Order,114-7324206-6909003,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,0.00,0,-7.15,-10.40,0.00,0.0,0.0,119.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2023-01-31 16:10:50-08:00,17315251481,Order,111-6292348-3459466,i1305JF-PJ-US,ALLDOCUBE GTBook 13 Windows 11 Laptop 13.5 inc...,1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,0.00,0,-18.36,-24.48,-7.92,0.0,0.0,273.59
505,2023-01-31 18:44:15-08:00,17315251481,Order,112-5898200-4188202,T1030-64GB-PJUS,"ALLDOCUBE 4G LTE Tablet 10.4 Inch Android 12, ...",1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,-10.00,0,-8.18,-8.72,-5.80,0.0,0.0,94.48
506,2023-01-31 20:02:02-08:00,17315251481,Order,113-7809224-4840228,T1030-64GB-PJUS,"ALLDOCUBE 4G LTE Tablet 10.4 Inch Android 12, ...",1.0,amazon.com,Standard Orders,Amazon,...,0.0,0,0.00,0,-7.14,-9.52,-5.80,0.0,0.0,103.68
507,2023-01-31 22:46:11-08:00,17315251481,NaN,7e93ce80-72a3-479f-a8bb-cff802b845ef,NaN,Save $20 on on ALLDOCUBE iwork 20 Pro,1.0,Amazon.com,Standard Orders,NaN,...,0.0,0,0.00,0,0.00,0.00,0.00,-0.6,0.0,-0.60


#### <a name="2.2"></a> 加拿大

In [5]:
list_cols_ca = ["date_time", "settlement_id", "type", "order_id", "sku", "description", "quantity", "marketplace", 
                "fulfillment", "order_city", "order_state", "order_postal", "tax_collection_model", 
                "product_sales", "product_sales_tax", "shipping_credits", "shipping_credits_tax", "gift_wrap_credits", 
                "giftwrap_credits_tax", "Regulatory_Fee", "Tax_On_Regulatory_Fee", "promotional_rebates", 
                "promotional_rebates_tax", "marketplace_withheld_tax", "selling_fees", "fba_fees", 
                "other_transaction_fees", "other", "total"]

date_parser_ca = lambda x: dateparser.parse(x, settings={'DATE_ORDER': 'MDY'})

df_ca_bcrh = pd.read_csv(list_csv[0], names=list_cols_ca, skiprows=8, thousands=',', decimal='.', 
                         parse_dates=['date_time'], date_parser=date_parser_ca)
df_ca_lx = pd.read_csv(list_csv[3], names=list_cols_ca, skiprows=8, thousands=',', decimal='.',
                       parse_dates=['date_time'], date_parser=date_parser_ca)

df_ca = pd.concat([df_ca_bcrh, df_ca_lx])
df_ca

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,Regulatory_Fee,Tax_On_Regulatory_Fee,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2023-01-01 01:09:07-08:00,17196097171,NaN,f2ded896-67fa-42ee-9341-c1ec388fb5be,NaN,Save C$40 on GT Book 15 laptop,8.0,Amazon.ca,NaN,NaN,...,0,0,0.0,0,0.0,0.00,0.00,-4.8,0.00,-4.80
1,2023-01-01 17:49:39-08:00,17196097171,Order,701-8015439-6705839,T1020HP-CA,"Android 11 Tablet 10.4 inch, ALLDOCUBE iPlay 4...",1.0,amazon.ca,Amazon,Puslinch,...,0,0,-30.0,0,-35.1,-16.20,-7.74,0.0,0.00,246.05
2,2023-01-01 20:53:12-08:00,17196097171,Order,702-6744322-9659446,i1502-CA,"ALLDOCUBE GT Book 15 Laptop 15.6 Inch, Windows...",1.0,amazon.ca,Amazon,new minas,...,0,0,-40.0,0,-60.0,-24.00,-15.38,0.0,0.00,360.61
3,2023-01-01 21:01:48-08:00,17196097171,Order,702-1491034-7868219,T803-CA,"Android 11 Tablet 8 inch, ALLDOCUBE Smile 1 Ta...",1.0,amazon.ca,Amazon,Vaughan,...,0,0,-20.0,0,-16.9,-7.80,-7.74,0.0,0.00,114.45
4,2023-01-01 21:36:58-08:00,17196097171,Order,701-2939492-7174623,T1026-CCA,"Android 11 Tablet 10.4 inch, ALLDOCUBE Kpad Ta...",1.0,amazon.ca,Amazon,West Montrose,...,0,0,-20.0,0,-23.4,-10.80,-7.74,0.0,0.00,161.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-01-31 03:32:02-08:00,17370110291,Refund,702-3185304-6910605,SPEN-LX-IT,Touchscreen Pen for Alldocube WindowsTablet iW...,1.0,amazon.ca,Amazon,MISSISSAUGA,...,0,0,0.0,0,7.8,2.88,0.00,0.0,0.00,-57.11
237,2023-01-31 07:25:58-08:00,17370110291,Order,701-6141445-1704215,SPEN-LX-IT,Touchscreen Pen for Alldocube WindowsTablet iW...,1.0,amazon.ca,Amazon,Windsor,...,0,0,0.0,0,-7.8,-3.60,-5.92,0.0,0.00,50.47
238,2023-01-31 18:05:31-08:00,17370110291,Transfer,NaN,NaN,To account ending with: 285,NaN,NaN,NaN,NaN,...,0,0,0.0,0,0.0,0.00,0.00,0.0,-10034.09,-10034.09
239,2023-01-31 20:34:19-08:00,17370110291,Refund,701-5105949-1874612,T1023PEN-LX-US,Tablet Pen-Stylus Pen for Alldocube X Game And...,1.0,amazon.ca,Amazon,North Vancouver,...,0,0,0.0,0,4.8,1.92,0.00,0.0,0.00,-38.07


### <a name="3"></a> 三、日本站

[回到目录](#toc)

In [6]:
list_cols_jp = ["date_time", "settlement_id", "type", "order_id", "sku", "description", "quantity", "marketplace", 
                "fulfillment", "order_city", "order_state", "order_postal", "tax_collection_model", 
                "product_sales", "product_sales_tax", "shipping_credits", "shipping_credits_tax", "gift_wrap_credits", 
                "giftwrap_credits_tax", "Amazon_Point_Cost", "promotional_rebates", "promotional_rebates_tax", 
                "marketplace_withheld_tax", "selling_fees", "fba_fees", "other_transaction_fees", "other", "total"]

date_parser_jp = lambda x: dateparser.parse(x, settings={'DATE_ORDER': 'YMD'})

df_jp_bcrh = pd.read_csv(list_csv[1], names=list_cols_jp, skiprows=8, thousands=',', decimal='.', 
                         parse_dates=['date_time'], date_parser=date_parser_jp, encoding='cp932')
df_jp_odlf = pd.read_csv(list_csv[13], names=list_cols_jp, skiprows=8, thousands=',', decimal='.',
                         parse_dates=['date_time'], date_parser=date_parser_jp, encoding='cp932')
df_jp_pj = pd.read_csv(list_csv[15], names=list_cols_jp, skiprows=8, thousands=',', decimal='.', 
                       parse_dates=['date_time'], date_parser=date_parser_jp, encoding='cp932')
df_jp_qy = pd.read_csv(list_csv[17], names=list_cols_jp, skiprows=8, thousands=',', decimal='.',
                       parse_dates=['date_time'], date_parser=date_parser_jp, encoding='cp932')

df_jp = pd.concat([df_jp_bcrh, df_jp_odlf, df_jp_pj, df_jp_qy])
df_jp

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,giftwrap_credits_tax,Amazon_Point_Cost,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2023-01-01 03:33:26+09:00,11318831563,NaN,12efcaba-ba6e-4ba5-b76a-d31e41476fe4,NaN,Save \4000 on FF iPlay50 タブレット,9.0,Amazon.co.jp,NaN,NaN,...,0.0,0.0,0.0,0.0,0,0.0,0.0,-540.0,0.0,-540.0
1,2023-01-01 03:41:09+09:00,11318831563,NaN,f6e6e892-37b9-4368-8181-00980fdf8bf5,NaN,Save \5000 on Kpad タブレット,5.0,Amazon.co.jp,NaN,NaN,...,0.0,0.0,0.0,0.0,0,0.0,0.0,-300.0,0.0,-300.0
2,2023-01-01 03:51:58+09:00,11318831563,NaN,b522632e-7d8d-4971-a2fb-36be87b4c56c,NaN,Save \3800 on iPlay40 Pro タブレット,3.0,Amazon.co.jp,NaN,NaN,...,0.0,0.0,0.0,0.0,0,0.0,0.0,-180.0,0.0,-180.0
3,2023-01-01 03:52:57+09:00,11318831563,NaN,4b18a89d-9509-40dd-a51c-ed56a1ee1d7c,NaN,Save \7000 on FF ALLDOCUBE X Game タブレット,2.0,Amazon.co.jp,NaN,NaN,...,0.0,0.0,0.0,0.0,0,0.0,0.0,-120.0,0.0,-120.0
4,2023-01-01 03:59:35+09:00,11318831563,NaN,acedda20-5c01-4780-af39-32def7f93ec6,NaN,表示価格より \4000 OFF iPlay50 タブレット,1.0,Amazon.co.jp,NaN,NaN,...,0.0,0.0,0.0,0.0,0,0.0,0.0,-60.0,0.0,-60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023-01-31 18:25:43+09:00,11337799313,注文,503-6345109-0659046,i1115-16GB-QYJP,"ALLDOCUBE IWORK GT, 2in1 ノートパソコン, 11"" Windows ...",1.0,amazon.jp,Amazon,NaN,...,0.0,0.0,0.0,0.0,0,-6400.0,0.0,0.0,0.0,73599.0
871,2023-01-31 18:31:07+09:00,11337799313,注文,503-3177401-6203055,iwork20Pro-Win11-JP,ALLDOCUBE 2 in1 ノートPC タッチスクリーン/タブレット 10.5インチキー...,1.0,amazon.jp,Amazon,NaN,...,0.0,-1260.0,-373.0,-37.0,0,-3259.0,0.0,0.0,0.0,37480.0
872,2023-01-31 18:59:21+09:00,11337799313,注文,250-0463742-5184650,T1030-128gb-JPQY,"【NEWモデル】ALLDOCUBE iplay50 タブレット Android 12, 4G...",1.0,amazon.jp,Amazon,NaN,...,0.0,-777.0,-4545.0,-455.0,0,-1610.0,-318.0,0.0,0.0,18195.0
873,2023-01-31 19:15:38+09:00,11337799313,返金,249-1926663-9567030,i1115-16GB-QYJP,"ALLDOCUBE IWORK GT, 2in1 ノートパソコン, 11"" Windows ...",1.0,amazon.jp,Amazon,NaN,...,0.0,0.0,0.0,0.0,0,5900.0,0.0,0.0,0.0,-74099.0


### <a name="4"></a> 四、欧洲站

[回到目录](#toc)

In [7]:
# 统一预处理
list_cols_eu = ["date_time", "settlement_id", "type", "order_id", "sku", "description", "quantity", "marketplace", 
                "fulfillment", "order_city", "order_state", "order_postal", "tax_collection_model", 
                "product_sales", "product_sales_tax", "shipping_credits", "shipping_credits_tax", "gift_wrap_credits", 
                "giftwrap_credits_tax", "promotional_rebates", "promotional_rebates_tax", "marketplace_withheld_tax", 
                "selling_fees", "fba_fees", "other_transaction_fees", "other", "total"]

date_parser_eu = lambda x: dateparser.parse(x, settings={'DATE_ORDER': 'DMY'})

#### <a name="4.1"></a> 德国站

In [8]:
df_de_lx = pd.read_csv(list_csv[4], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                       parse_dates=['date_time'], date_parser=date_parser_eu)
df_de_odlf = pd.read_csv(list_csv[9], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                         parse_dates=['date_time'], date_parser=date_parser_eu)

df_de = pd.concat([df_de_lx, df_de_odlf])
df_de

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,gift_wrap_credits,giftwrap_credits_tax,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2022-12-31 23:08:20+00:00,19016817262,NaN,f9ceab34-93d0-4e7f-ad19-138834fbcd02,NaN,Sparen Sie €30 bei iPlay40H Android 11 Tablet,1.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
1,2022-12-31 23:36:12+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,3.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-1.50,0.0,-1.50
2,2023-01-01 19:44:14+00:00,19016817262,Servicegebühr,NaN,NaN,Werbekosten,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-504.47,0.0,-504.47
3,2023-01-01 22:35:19+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,1.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
4,2023-01-01 22:43:33+00:00,19016817262,NaN,ff3b2c14-8d0f-440d-a384-b874ad695eaf,NaN,Sparen Sie €20 bei iplay50 tablet DE,6.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-3.00,0.0,-3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,2023-01-31 21:33:15+00:00,19267889922,Bestellung,304-7842609-0269162,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet, 10,36 Zoll 2000x120...",1.0,amazon.de,Amazon,Groß Schwiesow,...,0.0,0.0,0.00,0.00,-25.54,-11.20,-2.82,0.00,0.0,120.43
651,2023-01-31 21:51:22+00:00,19267889922,Bestellung,S02-6489846-6931124,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet, 10,36 Zoll 2000x120...",1.0,si-prod-marketplace-de.stores.amazon.de,Amazon,Düren,...,0.0,0.0,0.00,0.00,0.00,0.00,-7.60,0.00,0.0,-7.60
652,2023-01-31 21:52:19+00:00,19267889922,Bestellung,304-4627427-5367563,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Laptop, 15.6 Zoll FHD IPS ...",1.0,amazon.de,Amazon,Bad Camberg,...,0.0,0.0,-0.84,-0.16,-54.28,-23.80,-6.05,0.00,0.0,255.86
653,2023-01-31 22:30:38+00:00,19267889922,Bestellung,303-5230495-7561144,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet, 10,36 Zoll 2000x120...",1.0,amazon.de,Amazon,Gampern,...,0.0,0.0,0.00,0.00,-26.89,-11.29,-2.82,0.00,0.0,120.34


#### <a name="4.2"></a> 西班牙

In [9]:
df_es_lx = pd.read_csv(list_csv[5], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                       parse_dates=['date_time'], date_parser=date_parser_eu)
df_es_odlf = pd.read_csv(list_csv[10], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                         parse_dates=['date_time'], date_parser=date_parser_eu)

df_es = pd.concat([df_es_lx, df_es_odlf])
df_es

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,gift_wrap_credits,giftwrap_credits_tax,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2023-01-02 02:40:34+00:00,19045840722,Tarifa de prestación de servicio,NaN,NaN,Gastos de publicidad,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-119.44,0.0,-119.44
1,2023-01-02 04:04:09+00:00,19045840722,Reembolso,405-7720092-0304349,T1023-LX-IT,Tablet Android 11-ALLDOCUBE Tablet 10.5 Pulgad...,1.0,amazon.es,Amazon,Madrid,...,0.0,0.0,33.06,6.94,41.65,13.84,0.00,0.00,0.0,-184.50
2,2023-01-02 13:06:51+00:00,19045840722,Pedido,404-8536543-8829915,T1030-464EU-LX,"Android 12 Tablet 10.4"",ALLDOCUBE iPlay50 Tabl...",1.0,amazon.es,Amazon,Corroios,...,0.0,0.0,-16.26,-3.74,-26.67,-10.28,-5.95,0.00,0.0,99.73
3,2023-01-02 14:31:58+00:00,19045840722,Pedido,406-2269312-7021100,i1025-LX-IT,ALLDOCUBE PC Tablet 2 en 1 Tableta PC Laptop c...,1.0,amazon.es,Amazon,Vinaros,...,0.0,0.0,-16.53,-3.47,-59.01,-24.51,-10.94,0.00,0.0,245.53
4,2023-01-02 17:40:40+00:00,19045840722,Pedido,402-6540752-6243548,i1025-LX-IT,ALLDOCUBE PC Tablet 2 en 1 Tableta PC Laptop c...,1.0,amazon.es,Amazon,FARO-Montenegro,...,0.0,0.0,0.00,0.00,-68.43,-26.38,-10.94,0.00,0.0,260.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,2023-01-31 02:59:05+00:00,19267841382,Pedido,402-7999490-7858725,T1020HP-EU,ALLDOCUBE iPlay40 Pro Tableta de 10.4 Pulgadas...,1.0,amazon.es,Amazon,LA LINEA DE LA CONCEPCION,...,0.0,0.0,-16.53,-3.47,-41.65,-17.30,-3.58,0.00,0.0,177.46
149,2023-01-31 05:53:48+00:00,19267841382,Pedido,403-1858736-8774732,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tableta de 10,36 Pulgadas, ...",1.0,amazon.es,Amazon,Blanes,...,0.0,0.0,0.00,0.00,-34.71,-14.42,-3.58,0.00,0.0,147.28
150,2023-01-31 10:14:57+00:00,19267841382,Pedido,402-7247769-9234700,T1020HP-EU,ALLDOCUBE iPlay40 Pro Tableta de 10.4 Pulgadas...,1.0,amazon.es,Amazon,NAVALMORAL DE LA MATA,...,0.0,0.0,-16.53,-3.47,-41.65,-17.30,-3.58,0.00,0.0,177.46
151,2023-01-31 13:23:56+00:00,19267841382,Tarifa de prestación de servicio,NaN,NaN,Gastos de publicidad,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-518.52,0.0,-518.52


#### <a name="4.3"></a> 法国站

统一规范千分位：

In [10]:
with open(list_csv[6], 'rb') as f:
    file_data = f.read()
    file_encoding = chardet.detect(file_data)['encoding']
file_encoding

'UTF-8-SIG'

In [11]:
with open(list_csv[6], 'r', encoding=file_encoding) as f:
    file_fr_lx = f.read().replace('\u202f', '.')
    
with open(list_csv[11], 'r', encoding=file_encoding) as f:
    file_fr_odlf = f.read().replace('\u202f', '.')

In [12]:
df_fr_lx = pd.read_csv(io.StringIO(file_fr_lx), names=list_cols_eu, skiprows=8, 
                       thousands='.', decimal=',', 
                       parse_dates=['date_time'], date_parser=date_parser_eu)
df_fr_odlf = pd.read_csv(io.StringIO(file_fr_odlf), names=list_cols_eu, skiprows=8, 
                         thousands='.', decimal=',', 
                         parse_dates=['date_time'], date_parser=date_parser_eu)

df_fr = pd.concat([df_fr_lx, df_fr_odlf])
df_fr

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,gift_wrap_credits,giftwrap_credits_tax,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2022-12-31 23:29:14+00:00,19045860692,NaN,eb34ec15-68ba-4af2-bb35-79557c13129e,NaN,Économisez €20 sur iPlay50 FR,1.0,Amazon.fr,NaN,NaN,...,0,0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
1,2023-01-01 22:38:57+00:00,19045860692,NaN,5d60e0a9-eaa7-458b-9b41-a0a0f7b61bf6,NaN,Économisez €30 sur iWork 20 Pro Windows 11 Ta...,1.0,Amazon.fr,NaN,NaN,...,0,0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
2,2023-01-02 02:50:05+00:00,19045860692,Frais de service,NaN,NaN,Prix de la publicité,NaN,NaN,NaN,NaN,...,0,0,0.00,0.00,0.00,0.00,0.00,-200.63,0.0,-200.63
3,2023-01-02 16:07:47+00:00,19045860692,Commande,407-2545975-5794734,i1025-LX-IT,ALLDOCUBE 2 en 1 Tablette PC Portable Tablette...,1.0,amazon.fr,Amazon,LEVALLOIS PERRET,...,0,0,0.00,0.00,-61.67,-26.68,-6.61,0.00,0.0,275.03
4,2023-01-02 20:49:00+00:00,19045860692,Commande,403-9040622-8277925,T1030-464EU-LX,"Tablette Android 12 10,4"",ALLDOCUBE iPlay50 Ta...",1.0,amazon.fr,Amazon,Clermont Ferrand,...,0,0,-16.67,-3.33,-23.34,-10.09,-4.71,0.00,0.0,101.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2023-01-30 17:11:29+00:00,19190298592,Commande,171-9092959-8405127,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Ordinateur Portable, 15.6 ...",1.0,amazon.fr,Amazon,Sollies Pont,...,0,0,-41.67,-8.33,-58.34,-25.23,-9.44,0.00,0.0,256.98
139,2023-01-30 22:50:18+00:00,19190298592,NaN,e5e503cb-05fa-4b8a-bdef-36dd32c6ad3f,NaN,Save €45 on ALLDOCUBE Smile X,1.0,Amazon.fr,NaN,NaN,...,0,0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
140,2023-01-31 14:37:09+00:00,19190298592,Remboursement,408-4363420-2384342,i1405-EU,ALLDOCUBE GTBook Ordinateur Portable 14 Pouces...,1.0,amazon.fr,Amazon,Guer,...,0,0,2.07,0.42,55.25,19.12,0.00,0.00,0.0,-257.12
141,2023-01-31 15:23:47+00:00,19190298592,Remboursement,405-4741359-4511513,i1115-XFJP,ALLDOCUBE Clavier haut de gamme pour tablette ...,1.0,amazon.fr,Amazon,Six Fours les plages,...,0,0,0.00,0.00,13.33,9.89,0.00,0.00,0.0,-56.77


#### <a name="4.4"></a> 意大利

In [13]:
df_it_lx = pd.read_csv(list_csv[7], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                       parse_dates=['date_time'], date_parser=date_parser_eu)
df_it_odlf = pd.read_csv(list_csv[12], names=list_cols_eu, skiprows=8, thousands='.', decimal=',', 
                         parse_dates=['date_time'], date_parser=date_parser_eu)

df_it = pd.concat([df_it_lx, df_it_odlf])
df_it

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,gift_wrap_credits,giftwrap_credits_tax,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2023-01-02 00:50:13+00:00,19045855262,Ordine,171-9900004-6239503,T1030-464EU-LX,"Android 12 Tablet 10,4 Pollici,ALLDOCUBE iPlay...",1.0,amazon.it,Amazon,Gela,...,0,0,-19.66,-4.33,-25.24,-10.09,-4.13,0.0,0.0,100.53
1,2023-01-02 02:59:08+00:00,19045855262,Commissione di servizio,NaN,NaN,Costo della pubblicità,NaN,NaN,NaN,NaN,...,0,0,0.00,0.00,0.00,0.00,0.00,-418.2,0.0,-418.20
2,2023-01-02 09:07:38+00:00,19045855262,Rimborso,402-0799587-6581934,i1025-LX-IT,ALLDOCUBE PC Tablet Portatile 2 in 1 Tablet PC...,1.0,amazon.it,Amazon,MASSAROSA,...,0,0,0.00,0.00,0.00,20.96,0.00,0.0,0.0,-339.03
3,2023-01-02 13:05:23+00:00,19045855262,Ordine,402-5667850-7413135,SPEN-LX-IT,ALLDOCUBE Penna Touchscreen for iWork20 & iWor...,1.0,amazon.it,Amazon,Taormina,...,0,0,0.00,0.00,0.00,-6.18,-3.13,0.0,0.0,30.68
4,2023-01-02 13:05:23+00:00,19045855262,Ordine,402-5667850-7413135,i1025-LX-IT,ALLDOCUBE PC Tablet Portatile 2 in 1 Tablet PC...,1.0,amazon.it,Amazon,Taormina,...,0,0,-20.00,0.00,0.00,-24.51,-5.88,0.0,0.0,309.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2023-01-30 21:38:24+00:00,19267760282,Ordine,407-9783233-0556345,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Portatili, 15.6 Pollici No...",1.0,amazon.it,Amazon,Pancalieri,...,0,0,0.00,0.00,-72.13,-28.84,-7.70,0.0,0.0,291.32
150,2023-01-30 21:40:00+00:00,19267760282,Rimborso,406-0484226-0468308,T1020HP-EU,"ALLDOCUBE iPlay40 Pro Tablet da 10,4 pollici, ...",1.0,amazon.it,Amazon,Cagliari,...,0,0,0.00,0.00,43.28,13.84,0.00,0.0,0.0,-182.87
151,2023-01-31 03:46:39+00:00,19267760282,Rimborso,402-5350686-8857902,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,Amazon,Trezzano sul Naviglio,...,0,0,24.59,5.41,30.65,9.81,0.00,0.0,0.0,-129.53
152,2023-01-31 12:36:12+00:00,19267760282,Ordine,402-3144544-5502763,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,Amazon,Manoppello,...,0,0,-33.99,0.00,0.00,-9.66,-3.87,0.0,0.0,120.40


### <a name="5"></a> 五、全站点整合

[回到目录](#toc)

In [14]:
# 欧洲四站合一
df_eu = pd.concat([df_de, df_es, df_fr, df_it])
df_eu

,date_time,settlement_id,type,order_id,sku,description,quantity,marketplace,fulfillment,order_city,...,gift_wrap_credits,giftwrap_credits_tax,promotional_rebates,promotional_rebates_tax,marketplace_withheld_tax,selling_fees,fba_fees,other_transaction_fees,other,total
0,2022-12-31 23:08:20+00:00,19016817262,NaN,f9ceab34-93d0-4e7f-ad19-138834fbcd02,NaN,Sparen Sie €30 bei iPlay40H Android 11 Tablet,1.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
1,2022-12-31 23:36:12+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,3.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-1.50,0.0,-1.50
2,2023-01-01 19:44:14+00:00,19016817262,Servicegebühr,NaN,NaN,Werbekosten,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-504.47,0.0,-504.47
3,2023-01-01 22:35:19+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,1.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
4,2023-01-01 22:43:33+00:00,19016817262,NaN,ff3b2c14-8d0f-440d-a384-b874ad695eaf,NaN,Sparen Sie €20 bei iplay50 tablet DE,6.0,Amazon.de,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,-3.00,0.0,-3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2023-01-30 21:38:24+00:00,19267760282,Ordine,407-9783233-0556345,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Portatili, 15.6 Pollici No...",1.0,amazon.it,Amazon,Pancalieri,...,0.0,0.0,0.00,0.00,-72.13,-28.84,-7.70,0.00,0.0,291.32
150,2023-01-30 21:40:00+00:00,19267760282,Rimborso,406-0484226-0468308,T1020HP-EU,"ALLDOCUBE iPlay40 Pro Tablet da 10,4 pollici, ...",1.0,amazon.it,Amazon,Cagliari,...,0.0,0.0,0.00,0.00,43.28,13.84,0.00,0.00,0.0,-182.87
151,2023-01-31 03:46:39+00:00,19267760282,Rimborso,402-5350686-8857902,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,Amazon,Trezzano sul Naviglio,...,0.0,0.0,24.59,5.41,30.65,9.81,0.00,0.00,0.0,-129.53
152,2023-01-31 12:36:12+00:00,19267760282,Ordine,402-3144544-5502763,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,Amazon,Manoppello,...,0.0,0.0,-33.99,0.00,0.00,-9.66,-3.87,0.00,0.0,120.40


字段统一：

In [15]:
# 北美
df_us['Amazon_Point_Cost'] = ""
df_ca['account_type'] = ""
df_ca['Amazon_Point_Cost'] = ""

# 日本
df_jp['account_type'] = ""
df_jp['Regulatory_Fee'] = ""
df_jp['Tax_On_Regulatory_Fee'] = ""

# 欧洲
df_eu['account_type'] = ""
df_eu['Regulatory_Fee'] = ""
df_eu['Tax_On_Regulatory_Fee'] = ""
df_eu['Amazon_Point_Cost'] = ""

In [16]:
common_cols = ["date_time", "settlement_id", "type", "order_id", "sku", "description", "quantity", "marketplace", 
               "account_type", "fulfillment", "order_city", "order_state", "order_postal", "tax_collection_model", 
               "product_sales", "product_sales_tax", "shipping_credits", "shipping_credits_tax", "gift_wrap_credits", 
               "giftwrap_credits_tax", "Regulatory_Fee", "Tax_On_Regulatory_Fee", "Amazon_Point_Cost", 
               "promotional_rebates", "promotional_rebates_tax", "marketplace_withheld_tax", "selling_fees", 
               "fba_fees", "other_transaction_fees", "other", "total"]

In [17]:
zhcn_cols = ['日期/时间', '费用编号', '费用类型', '订单号', 'SKU', '产品名称', '数量', '市场', 
             '账户类型', '配送渠道', '订单地址', '订单城市', '邮编', '税收方式', 
             '销售额', '销售税', '运费信用', '运费信用税', '礼品包装信用', 
             '礼包包装信用税', '监管费', '监管费税', '亚马逊积分', 
             '促销折扣', '促销折扣税', '市场预扣税', '销售费用', 
             'FBA配送费用', '其他费用', '其他', '总计']

In [18]:
df_us_zh = df_us[common_cols].rename(columns=dict(zip(common_cols, zhcn_cols)))
df_us_zh

,日期/时间,费用编号,费用类型,订单号,SKU,产品名称,数量,市场,账户类型,配送渠道,...,监管费税,亚马逊积分,促销折扣,促销折扣税,市场预扣税,销售费用,FBA配送费用,其他费用,其他,总计
0,2023-01-01 17:48:41-08:00,17196053011,Order,111-2105463-8945057,T1028-US2,"ALLDOCUBE Android 11 Tablet 10.1 inch, Smile X...",1.0,amazon.com,Standard Orders,Amazon,...,0,,-10.00,0,-8.40,-9.60,0.00,0.0,0.0,110.39
1,2023-01-01 21:41:05-08:00,17196053011,NaN,18179849-31f1-4131-a212-fff1511724ce,NaN,Save $20 on iPlay 50 tablet,8.0,Amazon.com,Standard Orders,NaN,...,0,,0.00,0,0.00,0.00,0.00,-4.8,0.0,-4.80
2,2023-01-01 22:39:46-08:00,17196053011,NaN,5da09c12-9b6a-42ed-8485-eb19be65fe3f,NaN,Save $20 on Smile X Tablet,1.0,Amazon.com,Standard Orders,NaN,...,0,,0.00,0,0.00,0.00,0.00,-0.6,0.0,-0.60
3,2023-01-01 22:40:41-08:00,17196053011,Order,111-6830145-0975449,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0,,-7.64,0,-12.35,-10.40,0.00,0.0,0.0,119.59
4,2023-01-01 23:15:44-08:00,17196053011,Order,114-7324206-6909003,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0,,0.00,0,-7.15,-10.40,0.00,0.0,0.0,119.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2023-01-31 16:10:50-08:00,17315251481,Order,111-6292348-3459466,i1305JF-PJ-US,ALLDOCUBE GTBook 13 Windows 11 Laptop 13.5 inc...,1.0,amazon.com,Standard Orders,Amazon,...,0,,0.00,0,-18.36,-24.48,-7.92,0.0,0.0,273.59
505,2023-01-31 18:44:15-08:00,17315251481,Order,112-5898200-4188202,T1030-64GB-PJUS,"ALLDOCUBE 4G LTE Tablet 10.4 Inch Android 12, ...",1.0,amazon.com,Standard Orders,Amazon,...,0,,-10.00,0,-8.18,-8.72,-5.80,0.0,0.0,94.48
506,2023-01-31 20:02:02-08:00,17315251481,Order,113-7809224-4840228,T1030-64GB-PJUS,"ALLDOCUBE 4G LTE Tablet 10.4 Inch Android 12, ...",1.0,amazon.com,Standard Orders,Amazon,...,0,,0.00,0,-7.14,-9.52,-5.80,0.0,0.0,103.68
507,2023-01-31 22:46:11-08:00,17315251481,NaN,7e93ce80-72a3-479f-a8bb-cff802b845ef,NaN,Save $20 on on ALLDOCUBE iwork 20 Pro,1.0,Amazon.com,Standard Orders,NaN,...,0,,0.00,0,0.00,0.00,0.00,-0.6,0.0,-0.60


In [19]:
df_ca_zh = df_ca[common_cols].rename(columns=dict(zip(common_cols, zhcn_cols)))
df_ca_zh

,日期/时间,费用编号,费用类型,订单号,SKU,产品名称,数量,市场,账户类型,配送渠道,...,监管费税,亚马逊积分,促销折扣,促销折扣税,市场预扣税,销售费用,FBA配送费用,其他费用,其他,总计
0,2023-01-01 01:09:07-08:00,17196097171,NaN,f2ded896-67fa-42ee-9341-c1ec388fb5be,NaN,Save C$40 on GT Book 15 laptop,8.0,Amazon.ca,,NaN,...,0,,0.0,0,0.0,0.00,0.00,-4.8,0.00,-4.80
1,2023-01-01 17:49:39-08:00,17196097171,Order,701-8015439-6705839,T1020HP-CA,"Android 11 Tablet 10.4 inch, ALLDOCUBE iPlay 4...",1.0,amazon.ca,,Amazon,...,0,,-30.0,0,-35.1,-16.20,-7.74,0.0,0.00,246.05
2,2023-01-01 20:53:12-08:00,17196097171,Order,702-6744322-9659446,i1502-CA,"ALLDOCUBE GT Book 15 Laptop 15.6 Inch, Windows...",1.0,amazon.ca,,Amazon,...,0,,-40.0,0,-60.0,-24.00,-15.38,0.0,0.00,360.61
3,2023-01-01 21:01:48-08:00,17196097171,Order,702-1491034-7868219,T803-CA,"Android 11 Tablet 8 inch, ALLDOCUBE Smile 1 Ta...",1.0,amazon.ca,,Amazon,...,0,,-20.0,0,-16.9,-7.80,-7.74,0.0,0.00,114.45
4,2023-01-01 21:36:58-08:00,17196097171,Order,701-2939492-7174623,T1026-CCA,"Android 11 Tablet 10.4 inch, ALLDOCUBE Kpad Ta...",1.0,amazon.ca,,Amazon,...,0,,-20.0,0,-23.4,-10.80,-7.74,0.0,0.00,161.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-01-31 03:32:02-08:00,17370110291,Refund,702-3185304-6910605,SPEN-LX-IT,Touchscreen Pen for Alldocube WindowsTablet iW...,1.0,amazon.ca,,Amazon,...,0,,0.0,0,7.8,2.88,0.00,0.0,0.00,-57.11
237,2023-01-31 07:25:58-08:00,17370110291,Order,701-6141445-1704215,SPEN-LX-IT,Touchscreen Pen for Alldocube WindowsTablet iW...,1.0,amazon.ca,,Amazon,...,0,,0.0,0,-7.8,-3.60,-5.92,0.0,0.00,50.47
238,2023-01-31 18:05:31-08:00,17370110291,Transfer,NaN,NaN,To account ending with: 285,NaN,NaN,,NaN,...,0,,0.0,0,0.0,0.00,0.00,0.0,-10034.09,-10034.09
239,2023-01-31 20:34:19-08:00,17370110291,Refund,701-5105949-1874612,T1023PEN-LX-US,Tablet Pen-Stylus Pen for Alldocube X Game And...,1.0,amazon.ca,,Amazon,...,0,,0.0,0,4.8,1.92,0.00,0.0,0.00,-38.07


In [20]:
df_jp_zh = df_jp[common_cols].rename(columns=dict(zip(common_cols, zhcn_cols)))
df_jp_zh

,日期/时间,费用编号,费用类型,订单号,SKU,产品名称,数量,市场,账户类型,配送渠道,...,监管费税,亚马逊积分,促销折扣,促销折扣税,市场预扣税,销售费用,FBA配送费用,其他费用,其他,总计
0,2023-01-01 03:33:26+09:00,11318831563,NaN,12efcaba-ba6e-4ba5-b76a-d31e41476fe4,NaN,Save \4000 on FF iPlay50 タブレット,9.0,Amazon.co.jp,,NaN,...,,0.0,0.0,0.0,0,0.0,0.0,-540.0,0.0,-540.0
1,2023-01-01 03:41:09+09:00,11318831563,NaN,f6e6e892-37b9-4368-8181-00980fdf8bf5,NaN,Save \5000 on Kpad タブレット,5.0,Amazon.co.jp,,NaN,...,,0.0,0.0,0.0,0,0.0,0.0,-300.0,0.0,-300.0
2,2023-01-01 03:51:58+09:00,11318831563,NaN,b522632e-7d8d-4971-a2fb-36be87b4c56c,NaN,Save \3800 on iPlay40 Pro タブレット,3.0,Amazon.co.jp,,NaN,...,,0.0,0.0,0.0,0,0.0,0.0,-180.0,0.0,-180.0
3,2023-01-01 03:52:57+09:00,11318831563,NaN,4b18a89d-9509-40dd-a51c-ed56a1ee1d7c,NaN,Save \7000 on FF ALLDOCUBE X Game タブレット,2.0,Amazon.co.jp,,NaN,...,,0.0,0.0,0.0,0,0.0,0.0,-120.0,0.0,-120.0
4,2023-01-01 03:59:35+09:00,11318831563,NaN,acedda20-5c01-4780-af39-32def7f93ec6,NaN,表示価格より \4000 OFF iPlay50 タブレット,1.0,Amazon.co.jp,,NaN,...,,0.0,0.0,0.0,0,0.0,0.0,-60.0,0.0,-60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2023-01-31 18:25:43+09:00,11337799313,注文,503-6345109-0659046,i1115-16GB-QYJP,"ALLDOCUBE IWORK GT, 2in1 ノートパソコン, 11"" Windows ...",1.0,amazon.jp,,Amazon,...,,0.0,0.0,0.0,0,-6400.0,0.0,0.0,0.0,73599.0
871,2023-01-31 18:31:07+09:00,11337799313,注文,503-3177401-6203055,iwork20Pro-Win11-JP,ALLDOCUBE 2 in1 ノートPC タッチスクリーン/タブレット 10.5インチキー...,1.0,amazon.jp,,Amazon,...,,-1260.0,-373.0,-37.0,0,-3259.0,0.0,0.0,0.0,37480.0
872,2023-01-31 18:59:21+09:00,11337799313,注文,250-0463742-5184650,T1030-128gb-JPQY,"【NEWモデル】ALLDOCUBE iplay50 タブレット Android 12, 4G...",1.0,amazon.jp,,Amazon,...,,-777.0,-4545.0,-455.0,0,-1610.0,-318.0,0.0,0.0,18195.0
873,2023-01-31 19:15:38+09:00,11337799313,返金,249-1926663-9567030,i1115-16GB-QYJP,"ALLDOCUBE IWORK GT, 2in1 ノートパソコン, 11"" Windows ...",1.0,amazon.jp,,Amazon,...,,0.0,0.0,0.0,0,5900.0,0.0,0.0,0.0,-74099.0


In [21]:
df_eu_zh = df_eu[common_cols].rename(columns=dict(zip(common_cols, zhcn_cols)))
df_eu_zh

,日期/时间,费用编号,费用类型,订单号,SKU,产品名称,数量,市场,账户类型,配送渠道,...,监管费税,亚马逊积分,促销折扣,促销折扣税,市场预扣税,销售费用,FBA配送费用,其他费用,其他,总计
0,2022-12-31 23:08:20+00:00,19016817262,NaN,f9ceab34-93d0-4e7f-ad19-138834fbcd02,NaN,Sparen Sie €30 bei iPlay40H Android 11 Tablet,1.0,Amazon.de,,NaN,...,,,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
1,2022-12-31 23:36:12+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,3.0,Amazon.de,,NaN,...,,,0.00,0.00,0.00,0.00,0.00,-1.50,0.0,-1.50
2,2023-01-01 19:44:14+00:00,19016817262,Servicegebühr,NaN,NaN,Werbekosten,NaN,NaN,,NaN,...,,,0.00,0.00,0.00,0.00,0.00,-504.47,0.0,-504.47
3,2023-01-01 22:35:19+00:00,19016817262,NaN,ffb378d9-0ffe-4d74-8a86-844a4e9e9878,NaN,Sparen Sie €30 bei ALLDOCUBE iWork20 Pro,1.0,Amazon.de,,NaN,...,,,0.00,0.00,0.00,0.00,0.00,-0.50,0.0,-0.50
4,2023-01-01 22:43:33+00:00,19016817262,NaN,ff3b2c14-8d0f-440d-a384-b874ad695eaf,NaN,Sparen Sie €20 bei iplay50 tablet DE,6.0,Amazon.de,,NaN,...,,,0.00,0.00,0.00,0.00,0.00,-3.00,0.0,-3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2023-01-30 21:38:24+00:00,19267760282,Ordine,407-9783233-0556345,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Portatili, 15.6 Pollici No...",1.0,amazon.it,,Amazon,...,,,0.00,0.00,-72.13,-28.84,-7.70,0.00,0.0,291.32
150,2023-01-30 21:40:00+00:00,19267760282,Rimborso,406-0484226-0468308,T1020HP-EU,"ALLDOCUBE iPlay40 Pro Tablet da 10,4 pollici, ...",1.0,amazon.it,,Amazon,...,,,0.00,0.00,43.28,13.84,0.00,0.00,0.0,-182.87
151,2023-01-31 03:46:39+00:00,19267760282,Rimborso,402-5350686-8857902,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,,Amazon,...,,,24.59,5.41,30.65,9.81,0.00,0.00,0.0,-129.53
152,2023-01-31 12:36:12+00:00,19267760282,Ordine,402-3144544-5502763,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,,Amazon,...,,,-33.99,0.00,0.00,-9.66,-3.87,0.00,0.0,120.40


In [22]:
df_all = pd.concat([df_us_zh, df_ca_zh, df_jp_zh, df_eu_zh])
df_all

,日期/时间,费用编号,费用类型,订单号,SKU,产品名称,数量,市场,账户类型,配送渠道,...,监管费税,亚马逊积分,促销折扣,促销折扣税,市场预扣税,销售费用,FBA配送费用,其他费用,其他,总计
0,2023-01-01 17:48:41-08:00,17196053011,Order,111-2105463-8945057,T1028-US2,"ALLDOCUBE Android 11 Tablet 10.1 inch, Smile X...",1.0,amazon.com,Standard Orders,Amazon,...,0,,-10.00,0.00,-8.40,-9.60,0.00,0.0,0.0,110.39
1,2023-01-01 21:41:05-08:00,17196053011,NaN,18179849-31f1-4131-a212-fff1511724ce,NaN,Save $20 on iPlay 50 tablet,8.0,Amazon.com,Standard Orders,NaN,...,0,,0.00,0.00,0.00,0.00,0.00,-4.8,0.0,-4.80
2,2023-01-01 22:39:46-08:00,17196053011,NaN,5da09c12-9b6a-42ed-8485-eb19be65fe3f,NaN,Save $20 on Smile X Tablet,1.0,Amazon.com,Standard Orders,NaN,...,0,,0.00,0.00,0.00,0.00,0.00,-0.6,0.0,-0.60
3,2023-01-01 22:40:41-08:00,17196053011,Order,111-6830145-0975449,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0,,-7.64,0.00,-12.35,-10.40,0.00,0.0,0.0,119.59
4,2023-01-01 23:15:44-08:00,17196053011,Order,114-7324206-6909003,T1030-US,"ALLDOCUBE Android 12 Tablet 10.4 inch, iPlay 5...",1.0,amazon.com,Standard Orders,Amazon,...,0,,0.00,0.00,-7.15,-10.40,0.00,0.0,0.0,119.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2023-01-30 21:38:24+00:00,19267760282,Ordine,407-9783233-0556345,i1502-ODLF-EU,"ALLDOCUBE GTBook 15 Portatili, 15.6 Pollici No...",1.0,amazon.it,,Amazon,...,,,0.00,0.00,-72.13,-28.84,-7.70,0.0,0.0,291.32
150,2023-01-30 21:40:00+00:00,19267760282,Rimborso,406-0484226-0468308,T1020HP-EU,"ALLDOCUBE iPlay40 Pro Tablet da 10,4 pollici, ...",1.0,amazon.it,,Amazon,...,,,0.00,0.00,43.28,13.84,0.00,0.0,0.0,-182.87
151,2023-01-31 03:46:39+00:00,19267760282,Rimborso,402-5350686-8857902,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,,Amazon,...,,,24.59,5.41,30.65,9.81,0.00,0.0,0.0,-129.53
152,2023-01-31 12:36:12+00:00,19267760282,Ordine,402-3144544-5502763,T1030-EU 6+128,"ALLDOCUBE iPlay 50 Tablet PC 10,36 pollici, An...",1.0,amazon.it,,Amazon,...,,,-33.99,0.00,0.00,-9.66,-3.87,0.0,0.0,120.40


In [23]:
df_us_zh.to_csv('Jan23US.csv')
df_ca_zh.to_csv('Jan23CA.csv')
df_jp_zh.to_csv('Jan23JP.csv')
df_eu_zh.to_csv('Jan23EU.csv')
df_all.to_csv('Jan23TEST.csv')

### <a name="6"></a> 六、利润模型构建

[回到目录](#toc)